In [25]:
import numpy as np
import torch
import pickle
import experiment_runner as er
from torch import nn
from sklearn.model_selection import train_test_split

features = pickle.load(open("../../inputs/texas_100_features.p", 'rb')).astype(np.float32)
labels = pickle.load(open("../../inputs/texas_100_labels.p", 'rb'))

ds = list(zip(features, labels))

_, ds = train_test_split(ds, shuffle=True)

texas_train, texas_test = train_test_split(ds, test_size=.3, shuffle=True)
print(len(texas_train), len(texas_test))


class Texas_Classifier(nn.Module):
    def __init__(self, w):
        super(Texas_Classifier, self).__init__()
        self.fc1 = nn.Linear(6169, w)
        self.fc2 = nn.Linear(w, w)
        self.fc3 = nn.Linear(w, 100)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        x = nn.ReLU()(x)
        x = self.fc3(x)
        return torch.log_softmax(x,dim=1)
    
    

model = Texas_Classifier(128)
info, model = er.run_experiment(model,
                                texas_train,
                                texas_test,
                                epsilon=0,
                                alpha=2,
                                epochs=8,
                                add_noise=False,
                                throw_out_threshold=False,
                                throw_out_std=0,
                                batch_size=64,
                                lf=torch.nn.NLLLoss,
                                print_rate=1)


11783 5050
Epoch 0: train loss 4.143445014953613, test loss 3.7030820846557617, adv 0.022762506967670015, acc 0.16806891025641027
Epoch 1: train loss 2.980689764022827, test loss 2.5911831855773926, adv 0.058517541109253, acc 0.36999198717948717
Epoch 2: train loss 2.0997700691223145, test loss 2.091527223587036, adv 0.0868520066889632, acc 0.4443108974358974
Epoch 3: train loss 1.6351245641708374, test loss 1.868760108947754, adv 0.1125104515050167, acc 0.48818108974358976
Epoch 4: train loss 1.3511989116668701, test loss 1.7969446182250977, adv 0.13864792363433676, acc 0.5130208333333334
Epoch 5: train loss 1.1503796577453613, test loss 1.7155781984329224, adv 0.17653898411371238, acc 0.5300480769230769
Epoch 6: train loss 0.9886560440063477, test loss 1.7079403400421143, adv 0.21239417851170567, acc 0.5248397435897436
Epoch 7: train loss 0.8645074367523193, test loss 1.73690664768219, adv 0.2334408967391305, acc 0.5322516025641025


In [26]:
from copy import deepcopy
import membership_inference as mi
train_loader = torch.utils.data.DataLoader(texas_train, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(texas_test, batch_size=64, shuffle=False, drop_last=True)

In [28]:
sens = .001 * 3.16 * 8 # lr * constant term * num epochs
alpha = 20
epsilon = 25
for e in [1, 5, 10, 20, 25, 50, 75, 100]:
    m1 = deepcopy(model)
    sigma = np.sqrt((sens**2 * alpha) / (2 * e))
    print(sigma)
    with torch.no_grad():
        for p in m1.parameters():
            p += (sigma * torch.randn(p.shape,device=torch.cuda.current_device()).float())

        
    avg_test_acc, avg_test_l = er.loader_accuracy(m1, test_loader, lf=torch.nn.NLLLoss())
    avg_train_l = 0.8645074367523193
            
    tpr = mi.run_yeom_loader(m1, avg_train_l, train_loader, lf=torch.nn.NLLLoss)
    fpr = mi.run_yeom_loader(m1, avg_train_l, test_loader, lf=torch.nn.NLLLoss)
    adv = tpr-fpr
    print(avg_test_acc, adv)

0.07994237924905663
0.3729967948717949 0.1308963907469342
0.03575131885679184
0.47836538461538464 0.20172493380713497
0.02528
0.5172275641025641 0.2160304312987737
0.01787565942839592
0.5190304487179487 0.22526041666666669
0.015988475849811327
0.5152243589743589 0.22356857929208473
0.011305559694238937
0.5230368589743589 0.22898158793199552
0.009230950835820399
0.5292467948717948 0.23430314590301
0.007994237924905664
0.5262419871794872 0.22940835772017842
